<a href="https://colab.research.google.com/github/homayun-chaab/kaggle-titanic/blob/main/kaggle_titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install opendatasets

In [ ]:
import opendatasets as od
import pandas as pd

od.download(
    "https://www.kaggle.com/competitions/titanic/data")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: homayunchaab
Your Kaggle Key: ··········


100%|██████████| 34.1k/34.1k [00:00<00:00, 19.7MB/s]


Extracting archive ./titanic/titanic.zip to ./titanic


In [ ]:
# reading the XLSX file
file =('titanic/\
test.csv')
train_file =('titanic/\
test.csv')
nd = pd.read_csv(file)
test_pd = pd.read_csv(train_file)
# displaying the contents of the XLSX file

test_pd.head(2)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S


In [ ]:
def clean(data):
  data = data.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis=1)

  cols = ['SibSp', 'Parch', 'Fare', 'Age']
  for col in cols:
    data[col].fillna(data[col].median(), inplace=True)

  data.Embarked.fillna("U", inplace=True)
  return data


In [ ]:
data = clean(nd)
test = clean(test_pd)


In [ ]:
test.head(5)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,34.5,0,0,7.8292,Q
1,3,female,47.0,1,0,7.0000,S
2,2,male,62.0,0,0,9.6875,Q
3,3,male,27.0,0,0,8.6625,S
4,3,female,22.0,1,1,12.2875,S


In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

cols = ["Sex", "Embarked"]
for col in cols:
  data[col] = le.fit_transform(data[col])
  test[col] = le.fit_transform(test[col])
  print(le.classes_)

data.head(5)

['female' 'male']
['C' 'Q' 'S']


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,34.5,0,0,7.8292,1
1,3,0,47.0,1,0,7.0000,2
2,2,1,62.0,0,0,9.6875,1
3,3,1,27.0,0,0,8.6625,2
4,3,0,22.0,1,1,12.2875,2


In [ ]:
import torch
train = torch.tensor(data.values)
test = torch.tensor(test.values)

In [ ]:
train.shape

torch.Size([418, 7])

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
train = train.type(torch.float)
test = test.type(torch.float)
train = train.to(device)
test = test.to(device)

In [ ]:
train.dtype

torch.float32

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train,
                                                    data,
                                                    test_size=0.2,
                                                    random_state=42)
y_train = torch.tensor(y_train.values)
y_test = torch.tensor(y_test.values)

In [ ]:
y_train = y_train.type(torch.float)
y_test = y_test.type(torch.float)

In [ ]:
from torch import nn
class titanikV1(nn.Module):
  def __init__(self,
               input_shape: int,
               output_shape: int,
               hidden_units: int):
    super().__init__()
    self.linear_layer_stack = nn.Sequential(
        nn.Linear(in_features=input_shape, out_features=hidden_units),
        nn.ReLU(),
        nn.Linear(in_features=hidden_units, out_features=hidden_units),
        nn.ReLU(),
        nn.Linear(in_features=hidden_units, out_features=output_shape),
    )
  def forward(self, x):
    return self.linear_layer_stack(x)


model_0 = titanikV1(input_shape=7,
                    output_shape=2,
                    hidden_units=10).to(device)




In [ ]:
model_0

titanikV1(
  (linear_layer_stack): Sequential(
    (0): Linear(in_features=7, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): ReLU()
    (4): Linear(in_features=10, out_features=2, bias=True)
  )
)

In [ ]:
model_0.state_dict()

OrderedDict([('linear_layer_stack.0.weight',
              tensor([[ 0.2890,  0.3137, -0.0885,  0.3472, -0.0828,  0.0763, -0.1840],
                      [ 0.2220,  0.3332, -0.2773,  0.3285,  0.0707,  0.2792,  0.0512],
                      [ 0.1822, -0.0534,  0.2914,  0.0559, -0.1764,  0.0963, -0.1741],
                      [-0.0443, -0.1535,  0.2507, -0.2984, -0.1742, -0.1067, -0.2273],
                      [ 0.0357, -0.3733,  0.3413, -0.3211,  0.2918,  0.0629, -0.1227],
                      [ 0.2336,  0.0589,  0.3054,  0.0413, -0.1192,  0.1016, -0.1025],
                      [ 0.1591,  0.3375,  0.2185, -0.1652,  0.2182,  0.0676,  0.1919],
                      [-0.2304, -0.3742, -0.1460, -0.2899,  0.3101,  0.1089,  0.1566],
                      [ 0.1195, -0.0066,  0.2958, -0.2685,  0.0238, -0.2580,  0.1165],
                      [-0.1302,  0.1158, -0.0787,  0.3135, -0.2240, -0.2254, -0.2254]],
                     device='cuda:0')),
             ('linear_layer_stack.0.bias',
 

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_0.parameters(),
                             lr=0.1)


In [ ]:
model_0(X_train)

tensor([[-3.4156e-02, -1.9911e+00],
        [ 1.3935e-02, -1.4657e+00],
        [-5.2601e-05, -1.6492e+00],
        [ 5.0560e-01, -8.7561e-01],
        [ 2.8143e-02, -1.2959e+00],
        [ 1.8064e-02, -1.6369e+00],
        [-5.9661e-02, -1.6932e+00],
        [ 5.0525e-01, -2.3329e-02],
        [-8.3012e-03, -2.8988e+00],
        [ 5.2886e-02, -1.0609e+00],
        [ 1.9931e-02, -1.5609e+00],
        [ 6.2330e-01, -2.5202e-01],
        [-5.4873e-02, -2.0248e+00],
        [ 3.1010e-01, -7.6437e-01],
        [ 1.5768e-02, -1.6122e+00],
        [-3.9023e-02, -2.0795e+00],
        [ 2.4464e-02, -1.5084e+00],
        [ 1.5768e-02, -1.6122e+00],
        [ 3.2520e-02, -1.4419e+00],
        [ 8.1849e-03, -1.6765e+00],
        [ 2.6251e-02, -1.6048e+00],
        [-8.0623e-02, -2.2539e+00],
        [ 4.9494e-02, -1.1719e+00],
        [ 1.3687e-02, -1.4909e+00],
        [ 2.0977e-02, -1.5575e+00],
        [ 4.5189e-02, -1.3416e+00],
        [ 1.2882e-02, -1.6576e+00],
        [-1.3299e-01, -3.347

In [ ]:
model_0.eval()
with torch.inference_mode():
  y_logits = model_0(X_test).to(device)

y_logits[:10]

tensor([[ 0.0282, -1.4601],
        [ 1.1104, -2.1338],
        [ 0.0376, -1.3202],
        [-0.0160, -2.0477],
        [-0.0417, -2.3350],
        [-0.1765, -3.1605],
        [ 0.0167, -1.7296],
        [-0.1729, -3.1034],
        [ 0.0240, -1.3310],
        [ 0.0119, -1.6614]], device='cuda:0')

In [ ]:
y_pred_probs = torch.sigmoid(y_logits)
print(y_logits[:5])
print(y_pred_probs[:5])


tensor([[ 0.0282, -1.4601],
        [ 1.1104, -2.1338],
        [ 0.0376, -1.3202],
        [-0.0160, -2.0477],
        [-0.0417, -2.3350]], device='cuda:0')
tensor([[0.5070, 0.1885],
        [0.7522, 0.1059],
        [0.5094, 0.2108],
        [0.4960, 0.1143],
        [0.4896, 0.0883]], device='cuda:0')


In [ ]:
y_peds = torch.argmax(y_pred_probs, dim=1)
y_peds

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')

In [ ]:
import requests
from pathlib import Path

if Path("helper_functions.py").is_file():
  print(f"helper_function.py already exist, skiping download...")
else:
  print("Downloading...")
  request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
  with open("helper_functions.py", "wb") as f:
    f.write(request.content)

helper_function.py already exist, skiping download...


In [ ]:
from helper_functions import accuracy_fn

In [ ]:
y_train.shape

torch.Size([334, 7])

In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)


epochs = 1000
X_tran, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)
for epoch in range(epochs):
  model_0.train()

  y_logits = model_0(X_train)
  y_pred = torch.softmax(y_logits, dim=1).argmax(dim=1)

  #y_logits = torch.t(y_logits)
  print(f"logits:{y_logits.shape} | y_Train:{y_train.shape}")
  loss = loss_fn(y_logits, y_train)
  #acc = accuracy_fn(y_true=X_train,
  #                  y_pred=y_pred)


  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  model_0.eval()
  with torch.inference_mode():
    test_logits = model_0(X_test).squeeze()
    test_preds = torch.softmax(test_logits, dim=1).argmax(dim=1)
    #print(f"test LOgits:{test_logits.shape} | y_train:{y_train.shape}")
    test_loss = loss_fn(test_logits, y_test)

  if epoch % 10 == 0:
    print(f"Epoch: {epoch} | Loss: {loss:.4f} | TEst Loss: {test_loss}")

logits:torch.Size([334, 2]) | y_Train:torch.Size([334, 7])


RuntimeError: 0D or 1D target tensor expected, multi-target not supported

In [ ]:
model_0.eval()
with torch.inference_mode():
  y_logits = model_0(X_test).to(device)

y_logits[:10]




tensor([[ 0.0282, -1.4601],
        [ 1.1104, -2.1338],
        [ 0.0376, -1.3202],
        [-0.0160, -2.0477],
        [-0.0417, -2.3350],
        [-0.1765, -3.1605],
        [ 0.0167, -1.7296],
        [-0.1729, -3.1034],
        [ 0.0240, -1.3310],
        [ 0.0119, -1.6614]], device='cuda:0')

In [ ]:
y_pred_probs = torch.softmax(y_logits, dim=1)
print(y_logits[:5])
print(y_pred_probs[:5])

tensor([[ 0.0282, -1.4601],
        [ 1.1104, -2.1338],
        [ 0.0376, -1.3202],
        [-0.0160, -2.0477],
        [-0.0417, -2.3350]], device='cuda:0')
tensor([[0.8158, 0.1842],
        [0.9625, 0.0375],
        [0.7954, 0.2046],
        [0.8841, 0.1159],
        [0.9083, 0.0917]], device='cuda:0')


In [ ]:

y_peds = torch.argmax(y_pred_probs, dim=1)
y_peds
Pclass	Sex	Age	SibSp	Parch	Fare	Embarked

SyntaxError: invalid syntax (<ipython-input-62-e1f240c134f1>, line 3)

In [ ]:
class titanikV2(nn.Module):
  def __init__(self):
    super().__init__()
    self.pclass = nn.Parameter(torch.randn(1,
                                           requires_grad=True,
                                           dtype=torch.float))
    self.Sex = nn.Parameter(torch.randn(1,
                                           requires_grad=True,
                                           dtype=torch.float))
    self.Age = nn.Parameter(torch.randn(1,
                                           requires_grad=True,
                                           dtype=torch.float))
    self.SibSp = nn.Parameter(torch.randn(1,
                                           requires_grad=True,
                                           dtype=torch.float))
    self.Parch = nn.Parameter(torch.randn(1,
                                           requires_grad=True,
                                           dtype=torch.float))
    self.Fare = nn.Parameter(torch.randn(1,
                                           requires_grad=True,
                                           dtype=torch.float))
    self.Embarked = nn.Parameter(torch.randn(1,
                                           requires_grad=True,
                                           dtype=torch.float))
  def forward(self, x):
    return self.weights